In [68]:
import pandas as pd
import numpy as np
import altair as alt

In [69]:
df = pd.read_csv("sparser_candidates.csv")
# this .csv file needs the following field:
# "host speed": original Sparser total time  
# "passed": number of filtered records
# "number of candidates": number candidates
# "Sparser parse: sparser parse time
# "preprocess": dpu preprocess
# "memory copy": host->dpu
# "dpu launch"
# "copy back": copy results from dpu back time
# "host parse": dpu host parse time
# "total records": total records


In [70]:
df['dpu_total_process'] = df['preprocess'] + df['host parse']
df['dpu_total_memory'] = df['memory copy'] + df['copy back']
df['candidates_percentage'] = df['number of candidates'] / df['total records']
df['Sparser process'] = df['host speed'] - df['Sparser parse']

In [71]:
# sort based on candidate percentage
df = df.sort_values('candidates_percentage')

In [72]:
# create bunch of dataframes for later use
new_dpu_proc_df = df[['candidates_percentage', 'dpu_total_process']]
new_dpu_launch_df = df[['candidates_percentage', 'dpu launch']]
new_dpu_mem_df = df[['candidates_percentage', 'dpu_total_memory']]
new_host_df = df[['candidates_percentage', 'Sparser process']]
new_host_parse_df = df[['candidates_percentage', 'Sparser parse']]


In [73]:
new_dpu_proc_df.rename(columns={'dpu_total_process': 'vals'}, inplace =True)
new_dpu_launch_df.rename(columns={'dpu launch': 'vals'}, inplace =True)
new_dpu_mem_df.rename(columns={'dpu_total_memory': 'vals'}, inplace =True)
new_host_df.rename(columns={'Sparser process': 'vals'}, inplace =True)
new_host_parse_df.rename(columns={'Sparser parse': 'vals'}, inplace =True)

In [74]:
new_dpu_proc_df.insert(2, 'c2', len(new_dpu_proc_df) * ['dpu'], True)
new_dpu_launch_df.insert(2, 'c2', len(new_dpu_launch_df) * ['dpu'], True)
new_dpu_mem_df.insert(2, 'c2', len(new_dpu_mem_df) * ['dpu'], True)
new_host_df.insert(2, 'c2', len(new_host_df) * ['host'], True)
new_host_parse_df.insert(2, 'c2', len(new_host_parse_df) * ['host'], True)

new_dpu_proc_df.insert(3, 'name', len(new_dpu_proc_df) * ['dpu host parse'], True)
new_dpu_launch_df.insert(3, 'name', len(new_dpu_launch_df) * ['dpu launch'], True)
new_dpu_mem_df.insert(3, 'name', len(new_dpu_mem_df) * ['dpu memory transfer'], True)
new_host_df.insert(3, 'name', len(new_host_df) * ['Sparser process'], True)
new_host_parse_df.insert(3, 'name', len(new_host_parse_df) * ['Sparser parse'], True)


In [84]:
plot_df = pd.concat([new_dpu_proc_df, new_dpu_launch_df, new_dpu_mem_df, , new_host_parse_df, new_host_df], sort=False)
plot_df = plot_df.round({'candidates_percentage': 4})

SyntaxError: invalid syntax (<ipython-input-84-14ba2a556c10>, line 1)

In [85]:
print(plot_df)

   candidates_percentage       vals    c2                 name
0                 0.1000  10.160085   dpu       dpu host parse
1                 0.2500  24.101290   dpu       dpu host parse
2                 0.5000  48.884967   dpu       dpu host parse
3                 0.7500  72.905426   dpu       dpu host parse
4                 0.9981  97.386476   dpu       dpu host parse
0                 0.1000   3.836240   dpu           dpu launch
1                 0.2500   3.933260   dpu           dpu launch
2                 0.5000   3.825560   dpu           dpu launch
3                 0.7500   3.825290   dpu           dpu launch
4                 0.9981   3.731700   dpu           dpu launch
0                 0.1000   2.039261   dpu  dpu memory transfer
1                 0.2500   2.096089   dpu  dpu memory transfer
2                 0.5000   2.036131   dpu  dpu memory transfer
3                 0.7500   2.026166   dpu  dpu memory transfer
4                 0.9981   2.091779   dpu  dpu memory t

In [87]:
## obtained from stackoverflow with modifications
# Plot data with Altair
alt.Chart(plot_df).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('c2:N', title=None),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(vals):Q',
        axis=alt.Axis(
            grid=False,
            title="Excution Time")),
    # important to sort within each stacked bar chart
     order=alt.Order('vals:N', sort='descending'),
    
    # tell Altair which field to use to use as the set of columns to be  represented in each group
    column=alt.Column('candidates_percentage:N', title='candidates percentage'),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('name:N',
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['#96ceb4', '#ffcc5c','#ff6f69', '#3376FF', '#FF8C33'],
            ),
        ))\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0    
    )

alt.Chart(...)